In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [185]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,RandomizedSearchCV

from sklearn.metrics.pairwise import haversine_distances
from math import radians, cos, sin, asin, sqrt

from pprint import pprint
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score
from statsmodels.api import add_constant
from statsmodels.regression.linear_model import OLS
import eli5
from eli5 import show_weights,show_prediction
import shap

In [3]:
data = pd.read_csv('/kaggle/input/seoul-bike-trip-duration-prediction/For_modeling.csv')

In [4]:
data.shape

In [5]:
data.head()

In [6]:
data.drop(['Unnamed: 0'],axis=1,inplace=True)

In [7]:
data.isnull().sum()

In [8]:
pd.set_option('display.max_columns', None)
data.describe()

In [9]:
data.info()

# Exploratory Data Analysis

In [10]:
# Add a graph in each part
sns.boxplot(data["Duration"])

In [11]:
sns.displot(data["Duration"])

In [12]:
sns.displot(data['Distance'],kind='kde')

In [13]:
sns.boxplot(data['Distance'])

In [14]:
data['Snow'].value_counts()

In [15]:
sns.displot(data['Snow'])

In [16]:
data['Precip'].value_counts()

In [17]:
sns.displot(data['Precip'])

Snow and Precip can be dropped as more than 50 percent of values in this column are 0.

In [18]:
data['Solar'].value_counts()

In [19]:
data['Haversine'].value_counts()

In [20]:
sns.displot(data['Haversine'])

In [21]:
sns.boxplot(data['Haversine'])

In [22]:
sns.displot(data['Dust'])

In [23]:
sns.boxplot(data['Dust'])

In [24]:
sns.displot(data['Temp'])

In [25]:
sns.boxplot(data['Temp'])

In [26]:
sns.displot(data['Humid'])

In [27]:
sns.boxplot(data['Humid'])

In [28]:
data['Pmin'].value_counts().plot.bar(figsize=(20,10))

In [29]:
data['Dmin'].value_counts().plot.bar(figsize=(20,10))

In [30]:
data['Phour'].value_counts().plot.bar(figsize=(20,10))

In [31]:
data['Dhour'].value_counts().plot.bar(figsize=(20,10))

In [32]:
data['DDweek'].value_counts().plot.bar(figsize=(20,10))

In [33]:
data['PDweek'].value_counts().plot.bar(figsize=(20,10))

In [34]:
data['Pmonth'].value_counts().plot.bar(figsize=(20,10))

In [35]:
data['Dmonth'].value_counts().plot.bar(figsize=(20,10))

In [36]:
data['Pday'].value_counts().plot.bar(figsize=(20,10))

In [37]:
data['Dday'].value_counts().plot.bar(figsize=(20,10))

In [38]:
sns.displot(data['PLatd'])

In [39]:
sns.boxplot(data['PLatd'])

In [40]:
sns.displot(data['PLong'])

In [41]:
sns.displot(data['PLong'])

In [42]:
sns.boxplot(data['PLong'])

In [43]:
plt.figure(figsize=(20,20))
sns.heatmap(data.corr(), annot=False)
plt.show()

Dmonth,Dday,Dhour,Dweekl,Dlat,Dlong,Snow,Precip,Groundtemp

# Feature Selection and Feature Engineering

In [44]:
data[['PLatd','PLong']] = data[['PLong','PLatd']]
data[['DLatd','DLong']] = data[['DLong','DLatd']]

In [45]:
data.columns

In [46]:
data.drop(['DDweek','Temp','Snow','Precip','Dmonth','Dday','Dhour','PLatd','PLong','DLatd','DLong'],axis=1,inplace=True)
data = data.loc[data["Dust"] * data["Wind"] * data["Haversine"] * data["Solar"]!= 0.0]
data.reset_index(drop=True, inplace=True)

In [47]:
data_sample = data.sample(n=53609, replace=True,random_state=101)
data_sample.reset_index(drop=True,inplace=True)
data_sample

In [48]:
X = data_sample.drop(labels=["Duration"], axis = 1)
y = data_sample["Duration"]

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

In [50]:
X_train.reset_index(drop=True,inplace=True)
y_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

In [51]:
rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=10)
X_train_rfe = rfe.fit_transform(X_train,y_train)
X_test_rfe = rfe.transform(X_test)

In [52]:
sns.set()
plt.figure(figsize=(7,5))
sns.barplot(y = X_train.columns, x = max(rfe.ranking_) - rfe.ranking_);

In [53]:
feature_list_rfe = [col for i,col in enumerate(X_train.columns) if rfe.support_[i]]
feature_list_rfe

In [54]:
(X_train[feature_list_rfe].values == X_train_rfe).sum()

# Training Linear Regression, XGBoost and Random Forest Model 

**Linear Regression Model using Standard Scaler with Outlier Treatment**

In [203]:
X_train_rfecp = X_train[feature_list_rfe].copy()
y_train_cp = y_train.copy()
# scaler = StandardScaler()
# scaler.fit(X_train_rfecp)
# X_test_cp = scaler.transform(X_test[feature_list_rfe])
X_test_cp = add_constant(X_test[feature_list_rfe])

In [204]:
X_train_rfecp = add_constant(X_train_rfecp)
reg = OLS(y_train_cp,X_train_rfecp).fit()

In [205]:
reg.summary()

In [206]:
y_pred = reg.predict(X_test_cp)

In [207]:
print(f'Training score : {r2_score(y_train, reg.predict(X_train_rfecp))}')

print()
print('r2 score:', r2_score(y_test, y_pred))
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

In [226]:
delta = reg.predict(X_train_rfecp)-y_train

In [227]:
sns.displot(delta)

In [228]:
idx = delta[(delta>50)|(delta<-50)].index

In [229]:
len(idx)

In [230]:
X_train_rfecp2 = X_train_rfecp
y_train_cp2 = y_train_cp
X_train_rfecp2 = X_train_rfecp2[~X_train_rfecp2.index.isin(idx)].reset_index(drop=True)
y_train_cp2 = y_train_cp2[~y_train_cp2.index.isin(idx)].reset_index(drop=True)

In [231]:
y_train_cp

In [232]:
X_train_rfecp2

In [233]:
reg3 = OLS(y_train_cp2,X_train_rfecp2).fit()

In [234]:
reg3.summary()

In [235]:
X_test_cp = add_constant(X_test[feature_list_rfe])
y_pred3 = reg3.predict(X_test_cp)

In [236]:
print(f'Training score : {r2_score(y_train_cp2, reg3.predict(X_train_rfecp2))}')
print()
print('r2 score:', r2_score(y_test, y_pred3))
print('MAE:', mean_absolute_error(y_test, y_pred3))
print('MSE:', mean_squared_error(y_test, y_pred3))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred3)))

**Random Forest Regressor**

In [131]:
rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)

rf.fit(X_train_rfe, y_train)
y_hat_test = rf.predict(X_test_rfe)

# rf.fit(X_train,y_train)
# y_pred = rf.predict(X_test)

In [132]:
y_hat_train = rf.predict(X_train_rfe)

In [133]:
y_hat_train

In [134]:
print(f'Training score : {rf.score(X_train_rfe, y_train)}')

print()
print('r2 score:', r2_score(y_test, y_hat_test))
print('MAE:', mean_absolute_error(y_test, y_hat_test))
print('MSE:', mean_squared_error(y_test, y_hat_test))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_hat_test)))

In [135]:
plt.scatter(y_test,y_hat_test)
plt.xlabel('Actuals')
plt.ylabel('Predicted')

In [136]:
plt.figure(figsize = (7,5))
sns.distplot(y_test - y_hat_test)
plt.title("Error Rate Distribution");

In [137]:
# # plt.scatter(y_test,y_hat_test)
# plt.xlabel('Actual')
# plt.ylabel('Predicted')

**XGBoost Regressor**

In [138]:
xgb = XGBRegressor()
xgb.fit(X_train_rfe, y_train)
y_hat_test = xgb.predict(X_test_rfe)

In [139]:
print(f'Training score : {xgb.score(X_train_rfe, y_train)}')

print()
print('r2 score:', r2_score(y_test, y_hat_test))
print('MAE:', mean_absolute_error(y_test, y_hat_test))
print('MSE:', mean_squared_error(y_test, y_hat_test))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_hat_test)))

In [140]:
plt.figure(figsize=(7,5))
sns.barplot(x = rf.feature_importances_, y = feature_list_rfe)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features", size=14);

# Hyperparameter Tuning Using Random Search

In [141]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [142]:
params = {'bootstrap': [True, False],
 'max_depth': [10, 50, 100,None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 1000, 2000]}

In [143]:
# Use the random grid to search for best hyperparameters
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = params, n_iter = 1, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train_rfe, y_train)

In [144]:
rf_random.best_params_

# Explainable AI using ELI5 and SHAP

In [145]:
rf1 = RandomForestRegressor()
xgb1 = XGBRegressor()
rf1.fit(X_train,y_train)
xgb1.fit(X_train,y_train)

**ELI5**

In [148]:
import eli5
from eli5 import show_weights

In [153]:
show_weights(rf1, feature_names=list(X.columns),show=["feature_importances"])

In [163]:
show_weights(xgb1, feature_names=list(X.columns),show=["feature_importances"])

In [165]:
import random

In [181]:
eli5.show_prediction(xgb1, X_test.iloc[1,:], feature_names=list(X_test.columns), show_feature_values=True)

In [182]:
eli5.show_prediction(xgb1, X_test.iloc[2,:], feature_names=list(X_test.columns), show_feature_values=True)

In [183]:
eli5.show_prediction(xgb1, X_test.iloc[10,:], feature_names=list(X_test.columns), show_feature_values=True)

**SHAP**

In [155]:
shap_values = shap.TreeExplainer(rf1).shap_values(X_train[:10])

In [156]:
shap.summary_plot(shap_values, X_train[:10], plot_type="bar")

In [157]:
shap_values = shap.TreeExplainer(xgb1).shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar")

In [158]:
shap.summary_plot(shap_values, X_train)

In [159]:
shap.dependence_plot('Haversine', shap_values, X_train)

In [160]:
shap.dependence_plot('Distance', shap_values, X_train)

In [161]:
shap.dependence_plot('Dust', shap_values, X_train)

In [162]:
shap.dependence_plot('Pmin', shap_values, X_train)